In [1]:
DATABASE='googol'
HOST='127.0.0.1'
USERNAME='kenjiwang'
PASSWORD='5517007'
PORT='5432'

In [2]:
import csv
import requests
import lxml.html

import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from transformers.dynamic_module_utils import get_imports
from unittest.mock import patch
from numpy.linalg import norm
import numpy as np
import psycopg2

/Users/kenjiwang/anaconda3/envs/search-engine/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
conn = psycopg2.connect(database=DATABASE,
	host=HOST,
	user=USERNAME,
	password=PASSWORD,
	port=PORT
)

cursor = conn.cursor()

cursor.execute("CREATE TABLE IF NOT EXISTS websites (id bigserial PRIMARY KEY NOT NULL, url text NOT NULL, title text NOT NULL, description text NOT NULL, title_emb vector(1024), description_emb vector(1024))")

conn.commit()

In [4]:
model = SentenceTransformer('dunzhang/stella_en_1.5B_v5', trust_remote_code=True)

In [5]:
def index_page(i, domain):
	if('api' in domain): return
	
	print(f"{i}: Indexing {domain}")
	website = requests.get('https://'+domain)
	if 'text/html' not in website.headers['content-type']: return
	html = lxml.html.fromstring(website.text)

	title = html.find('.//title').text
	title_formatted = title.replace('\'', '\'\'')
	title_emb = model.encode([title], normalize_embeddings=True)[0].tolist()

	try:
		try:
			description = html.find('.//meta[@name="description"]').get("content")
		except:
			description = html.find('.//meta[@property="og:description"]').get("content")
	except:
		description = ""

	description_emb = model.encode([description], normalize_embeddings=True)[0].tolist()
	description_formatted = description.replace('\'', '\'\'')

	query = f"INSERT INTO websites(title, url, description, title_emb, description_emb) VALUES ('{title_formatted}', '{domain}', '{description_formatted}', '{title_emb}', '{description_emb}')"
	cursor.execute(query)
	conn.commit()
	

In [6]:
with open('domains.csv', mode ='r') as file:
	csvFile = csv.reader(file)
	
	next(csvFile)
	for i in range(0,200): index_page(i, next(csvFile)[1])


0: Indexing facebook.com
